インポート

In [13]:
# 必要なライブラリのインポート
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

スクレイピングするURL

In [14]:
# schooのURLを設定する
#カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/{}"
#category_url = url_content.format(1)

他のカテゴリから取得する

In [15]:
category = {
    "ビジネススキル": "business/",
    "思考術・自己啓発": "thinking/",
    "マネジメント": "management/",
    "マーケティング": "marketing/",
    "UI・UXデザイン": "web_design/",
    "デザイン": "graphic/"
}

In [16]:
categories_url = []

for key, value in category.items():
    url = url_content.format(value)
    categories_url.append(url)

In [17]:
print(categories_url)

['https://schoo.jp/class/category/business/', 'https://schoo.jp/class/category/thinking/', 'https://schoo.jp/class/category/management/', 'https://schoo.jp/class/category/marketing/', 'https://schoo.jp/class/category/web_design/', 'https://schoo.jp/class/category/graphic/']


複数ページからも取ってこれるようにする

In [18]:
#categories_urls =[]

#for categories_url_pre in categories_url:
#    print(categories_url_pre+"?page=2")


for文を回す

In [21]:
#d_listに空リストを作成する
d_list = []

# schooのURLを設定する
#カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/{}"

category = {
    "ビジネススキル": "business/",
    "思考術・自己啓発": "thinking/",
    "マネジメント": "management/",
    "マーケティング": "marketing/",
    "UI・UXデザイン": "web_design/",
    "デザイン": "graphic/"
}

categories_url = []

for key, value in category.items():
    url = url_content.format(value)
    categories_url.append(url)

    for category_url in categories_url:
        # レスポンスを取得する
        res = requests.get(category_url)

        # 取得結果をsoupに格納する
        soup = BeautifulSoup(res.text, 'html.parser', from_encoding='utf-8')

        #カテゴリとして講義内容を取ってくる
        contents = soup.find_all('div', class_='scrollContent listItem')

        # for文で各情報を取得する
        for content in contents:
            #講義内容の詳細へとべるリンク
            a_tag = content.find('a')
            url = a_tag.get('href')
            #使うときは以下の形でリンクに飛べる
            link = "https://schoo.jp"+url
            #講義大分類名
            big_title = content.find("h3", class_="title").text
            #関連ワード
            # 関連ワードを格納するリスト
            related_words = []

            # .find_all()で取得した結果をループ
            for words in content.find_all("a", class_="tag ga_link"):
                # 各要素からテキストを取得し、リストに追加
                related_words.append(words.text)
            # リスト内の各文字列から "n/" と空白を削除する
            related_words = [word.strip('\n ') for word in related_words]
            
            # linkにアクセスしてHTMLを取得
            detail = requests.get(link)
            # ステータスコードが200（成功）でない場合はエラーを出力して終了
            if detail.status_code != 200:
                print(f"Error: Failed to fetch the page. Status code: {detail.status_code}")
                exit()
            # 取得したHTMLをBeautifulSoupで解析
            soup_detail = BeautifulSoup(detail.text, 'html.parser')

            #コース概要
            #agenda_element = soup_detail.find("strong")
            # 要素が存在する場合は text 属性を取得、存在しない場合はデフォルトの値を使用
            #agenda = agenda_element.text if agenda_element else "Not available"


            #講師名
            teacher_name = soup_detail.find("h4", class_="p-courseTeachers__columnName").text
            #現職
            teacher_post = soup_detail.find("p", class_="p-courseTeachers__columnPost").text
            #経歴
            teacher_profile = soup_detail.find("p", class_="p-courseTeachers__columnProfile").text

            #講義詳細
            #講義リンク
            lecture_link_pre = soup_detail.find_all("li", class_="p-classes__item js-foldingByItemCountItem")
            # lecture_link_pre内の各要素からaタグを見つけてそのhref属性を取得
            lecture_links = [item.find('a').get('href') for item in lecture_link_pre]

            # 各要素に"https://schoo.jp"を前に追加して新しいリストに格納
            lecture_url = ["https://schoo.jp" + link for link in lecture_links]


            #講義タイトル
            middle_title_pre = soup_detail.find_all("h3", class_="p-classes__title")
            # middle_title_preから各要素のテキストを取得
            middle_titles = [middle_title_pre_1.text.strip() for middle_title_pre_1 in middle_title_pre]


            # 講義時間
            lecture_time_pre = soup_detail.find_all("span", class_="p-classes__onAirDuration")
            lecture_times = [lecture_time.text.strip() for lecture_time in lecture_time_pre]

            dic = {
                "link": link,
                "big_title": big_title,
                #"agenda": agenda,
                "lecture_link": lecture_url,
                "middle_titles": middle_titles,
                "lecture_time": lecture_times,
                "teacher_name": teacher_name,
                "teacher_post": teacher_post,
                "teacher_profile": teacher_profile,
                "related_words": related_words,
            }

            d_list.append(dic)

/Users/yakushijimana/anaconda3/envs/reskill/lib/python3.11/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [22]:
len(d_list)

882

In [23]:
max = range(0, len(contents))
df =pd.DataFrame(d_list)

In [24]:
df

,link,big_title,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,https://schoo.jp/course/7887,全1回禅でひも解く「怒り」のしずめかた,[https://schoo.jp/class/10786],[怒った自分を解放する],[60min],河口 智賢,禅僧／耕雲院副住職,曹洞宗耕雲院副住職／曹洞宗観音寺住職／NPO法人ぐんないや-織syoku-理事長／株式会社つ...,"[コミュニケーション術, 行動習慣]"
1,https://schoo.jp/course/7876,全1回結果を出す人のチームワーク術,[https://schoo.jp/class/10764],[数字を追うのに疲れたあなたへ〜仕事のやりがいを取り戻すための処方箋〜],[60min],加藤 芳久,経営コンサルタント,●「理念型育成®」を日本で初めて開発・体系化した人財育成の専門家。\n株式会社ファイブベイ、...,"[コミュニケーション術, ファシリテーション, マネジメント, リーダーシップ, 人材育成,..."
2,https://schoo.jp/course/7892,全1回頭がいい人に思わせるにはどうすればいいの,[https://schoo.jp/class/10800],[頭がいい人と思わせる質問の仕方],[30min],安田 正,（株）パンネーションズ・コンサルティング・グループ代表取締役,株式会社パンネーションズ・コンサルティング・グループ代表取締役\n一般社団法人コミュニケーシ...,"[コミュニケーション術, 行動習慣]"
3,https://schoo.jp/course/7893,全3回イラストでまなぶ 会計・財務のきほん,"[https://schoo.jp/class/10801, https://schoo.j...","[貸借対照表をブタの貯金箱で読み解く, 損益計算書を風船で表現する]",[60min],松本 めぐみ,Star Compass(株)代表／松本興産(株)取締役,Star Compass株式会社 代表取締役。\n松本興産株式会社 取締役。\n米半導体装置...,"[ビジネス教養, 経理, 会計・財務]"
4,https://schoo.jp/course/7873,全1回『うまく言葉にできない』がなくなる言語化力,[https://schoo.jp/class/10763],[『うまく言葉にできない』がなくなる言語化力],[60min],山口 拓朗,伝える力[話す・書く]研究所／山口拓朗ライティングサロン主宰,出版社で編集者・記者を務めたのちに独立。27年間で3700件以上の取材・執筆歴を誇る。現在は...,"[印象づくり, 思考法, 交渉術]"
...,...,...,...,...,...,...,...,...,...
877,https://schoo.jp/course/7300,全4回デザイン温故知新〜過去から現在、未来へ繋がるヒントを探る,"[https://schoo.jp/class/9425, https://schoo.jp...","[「これまでのデザイン」を理解するための基礎知識, 無印良品など「シンプルなデザイン」の原点...","[60min, 60min, 60min, 60min]",中村 将大,帝京平成大学 助教,帝京平成大学 助教。1983年福岡生まれ。2009年から2021年3月まで東洋美術学校専任講...,"[デザイン, 歴史, リベラルアーツ, デザイン力]"
878,https://schoo.jp/course/7344,全5回Figma入門,"[https://schoo.jp/class/9508, https://schoo.jp...","[Figma入門１～インストールして使ってみよう／カラースタイル、テキストスタイル～, Fi...","[30min, 45min, 45min, 30min, 25min]",山本 麻美,株式会社ロクナナ UI/UXデザイナー,インディーズ音楽プラットフォームを作りたい一心でWebを独学。企業からWeb制作の仕事を請け...,"[カンプ制作, スマホサイト制作, プロトタイピング, Figma]"
879,https://schoo.jp/course/7265,全1回超実践！ブランディングの現場,[https://schoo.jp/class/9350],[超実践！ブランディングの現場],[60min],西澤 明洋,ブランディングデザイナー,1976年滋賀県生まれ。株式会社エイトブランディングデザイン代表。 「ブランディングデザイン...,"[クリエイティブデザイン, ブランディング, デザイン経営]"
880,https://schoo.jp/course/7199,全2回人を動かすバナーデザインの原理原則,"[https://schoo.jp/class/9178, https://schoo.jp...","[人を動かすバナーデザインの原理原則, 実践で学ぶ バナーデザインの「フォントの極意」]","[60min, 60min]",カトウ ヒカル,Webデザイナー,Webデザイン事務所kanvasのWebデザイナー。\nこれまで100サイト以上の制作に関わ...,"[タイポグラフィ, デザイン, ロゴ制作, デザイン力]"


csvファイルに保存

In [25]:
# csvファイルに出力する
df.to_csv("schoo_v2.csv", index = False)

In [26]:
df.head(5)

,link,big_title,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,https://schoo.jp/course/7887,全1回禅でひも解く「怒り」のしずめかた,[https://schoo.jp/class/10786],[怒った自分を解放する],[60min],河口 智賢,禅僧／耕雲院副住職,曹洞宗耕雲院副住職／曹洞宗観音寺住職／NPO法人ぐんないや-織syoku-理事長／株式会社つ...,"[コミュニケーション術, 行動習慣]"
1,https://schoo.jp/course/7876,全1回結果を出す人のチームワーク術,[https://schoo.jp/class/10764],[数字を追うのに疲れたあなたへ〜仕事のやりがいを取り戻すための処方箋〜],[60min],加藤 芳久,経営コンサルタント,●「理念型育成®」を日本で初めて開発・体系化した人財育成の専門家。\n株式会社ファイブベイ、...,"[コミュニケーション術, ファシリテーション, マネジメント, リーダーシップ, 人材育成,..."
2,https://schoo.jp/course/7892,全1回頭がいい人に思わせるにはどうすればいいの,[https://schoo.jp/class/10800],[頭がいい人と思わせる質問の仕方],[30min],安田 正,（株）パンネーションズ・コンサルティング・グループ代表取締役,株式会社パンネーションズ・コンサルティング・グループ代表取締役\n一般社団法人コミュニケーシ...,"[コミュニケーション術, 行動習慣]"
3,https://schoo.jp/course/7893,全3回イラストでまなぶ 会計・財務のきほん,"[https://schoo.jp/class/10801, https://schoo.j...","[貸借対照表をブタの貯金箱で読み解く, 損益計算書を風船で表現する]",[60min],松本 めぐみ,Star Compass(株)代表／松本興産(株)取締役,Star Compass株式会社 代表取締役。\n松本興産株式会社 取締役。\n米半導体装置...,"[ビジネス教養, 経理, 会計・財務]"
4,https://schoo.jp/course/7873,全1回『うまく言葉にできない』がなくなる言語化力,[https://schoo.jp/class/10763],[『うまく言葉にできない』がなくなる言語化力],[60min],山口 拓朗,伝える力[話す・書く]研究所／山口拓朗ライティングサロン主宰,出版社で編集者・記者を務めたのちに独立。27年間で3700件以上の取材・執筆歴を誇る。現在は...,"[印象づくり, 思考法, 交渉術]"


取ってきたリンクごとにID（大分類ID）をつける

In [27]:
#d_listに空リストを作成する
d_list = []

# schooのURLを設定する
#カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/{}"

category = {
    "ビジネススキル": "business/",
    "思考術・自己啓発": "thinking/",
    "マネジメント": "management/",
    "マーケティング": "marketing/",
    "UI・UXデザイン": "web_design/",
    "デザイン": "graphic/"
}

url_content = "https://schoo.jp/class/category/{}"

categories_url = {}

for key, value in category.items():
    url = url_content.format(value)
    categories_url[key] = url

print(categories_url)


{'ビジネススキル': 'https://schoo.jp/class/category/business/', '思考術・自己啓発': 'https://schoo.jp/class/category/thinking/', 'マネジメント': 'https://schoo.jp/class/category/management/', 'マーケティング': 'https://schoo.jp/class/category/marketing/', 'UI・UXデザイン': 'https://schoo.jp/class/category/web_design/', 'デザイン': 'https://schoo.jp/class/category/graphic/'}


In [30]:
# d_listに空リストを作成する
d_list = []

# schooのURLを設定する
# カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/{}"

category = {
    "ビジネススキル": "business/",
    "思考術・自己啓発": "thinking/",
    "マネジメント": "management/",
    "マーケティング": "marketing/",
    "UI・UXデザイン": "web_design/",
    "デザイン": "graphic/"
}

# categories_urlを辞書型に変更
categories_url = {}

for key, value in category.items():
    url = url_content.format(value)
    categories_url[key] = url

    for category_name, category_url in categories_url.items():
        # レスポンスを取得する
        res = requests.get(category_url)

        # 取得結果をsoupに格納する
        soup = BeautifulSoup(res.text, 'html.parser', from_encoding='utf-8')

        # カテゴリとして講義内容を取ってくる
        contents = soup.find_all('div', class_='scrollContent listItem')

        # for文で各情報を取得する
        for content in contents:
            # 講義内容の詳細へとべるリンク
            a_tag = content.find('a')
            url = a_tag.get('href')
            # 使うときは以下の形でリンクに飛べる
            link = "https://schoo.jp" + url
            # 講義大分類名
            big_title = content.find("h3", class_="title").text
            # 関連ワード
            # 関連ワードを格納するリスト
            related_words = []

            # .find_all()で取得した結果をループ
            for words in content.find_all("a", class_="tag ga_link"):
                # 各要素からテキストを取得し、リストに追加
                related_words.append(words.text)
            # リスト内の各文字列から "n/" と空白を削除する
            related_words = [word.strip('\n ') for word in related_words]

            # linkにアクセスしてHTMLを取得
            detail = requests.get(link)
            # ステータスコードが200（成功）でない場合はエラーを出力して終了
            if detail.status_code != 200:
                print(f"Error: Failed to fetch the page. Status code: {detail.status_code}")
                exit()
            # 取得したHTMLをBeautifulSoupで解析
            soup_detail = BeautifulSoup(detail.text, 'html.parser')

            # コース概要
            # agenda_element = soup_detail.find("strong")
            # 要素が存在する場合は text 属性を取得、存在しない場合はデフォルトの値を使用
            # agenda = agenda_element.text if agenda_element else "Not available"

            # 講師名
            teacher_name = soup_detail.find("h4", class_="p-courseTeachers__columnName").text
            # 現職
            teacher_post = soup_detail.find("p", class_="p-courseTeachers__columnPost").text
            # 経歴
            teacher_profile = soup_detail.find("p", class_="p-courseTeachers__columnProfile").text

            # 講義詳細
            # 講義リンク
            lecture_link_pre = soup_detail.find_all("li", class_="p-classes__item js-foldingByItemCountItem")
            # lecture_link_pre内の各要素からaタグを見つけてそのhref属性を取得
            lecture_links = [item.find('a').get('href') for item in lecture_link_pre]

            # 各要素に"https://schoo.jp"を前に追加して新しいリストに格納
            lecture_url = ["https://schoo.jp" + link for link in lecture_links]

            # 講義タイトル
            middle_title_pre = soup_detail.find_all("h3", class_="p-classes__title")
            # middle_title_preから各要素のテキストを取得
            middle_titles = [middle_title_pre_1.text.strip() for middle_title_pre_1 in middle_title_pre]

            # 講義時間
            lecture_time_pre = soup_detail.find_all("span", class_="p-classes__onAirDuration")
            lecture_times = [lecture_time.text.strip() for lecture_time in lecture_time_pre]

            # カテゴリ名を追加した辞書
            dic = {
                "category": category_name,
                "link": link,
                "big_title": big_title,
                # "agenda": agenda,
                "lecture_link": lecture_url,
                "middle_titles": middle_titles,
                "lecture_time": lecture_times,
                "teacher_name": teacher_name,
                "teacher_post": teacher_post,
                "teacher_profile": teacher_profile,
                "related_words": related_words,
            }

            d_list.append(dic)


/Users/yakushijimana/anaconda3/envs/reskill/lib/python3.11/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [31]:
len(d_list)

882

In [32]:
max = range(0, len(contents))
df =pd.DataFrame(d_list)

In [33]:
df

,category,link,big_title,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,ビジネススキル,https://schoo.jp/course/7887,全1回禅でひも解く「怒り」のしずめかた,[https://schoo.jp/class/10786],[怒った自分を解放する],[60min],河口 智賢,禅僧／耕雲院副住職,曹洞宗耕雲院副住職／曹洞宗観音寺住職／NPO法人ぐんないや-織syoku-理事長／株式会社つ...,"[コミュニケーション術, 行動習慣]"
1,ビジネススキル,https://schoo.jp/course/7876,全1回結果を出す人のチームワーク術,[https://schoo.jp/class/10764],[数字を追うのに疲れたあなたへ〜仕事のやりがいを取り戻すための処方箋〜],[60min],加藤 芳久,経営コンサルタント,●「理念型育成®」を日本で初めて開発・体系化した人財育成の専門家。\n株式会社ファイブベイ、...,"[コミュニケーション術, ファシリテーション, マネジメント, リーダーシップ, 人材育成,..."
2,ビジネススキル,https://schoo.jp/course/7892,全1回頭がいい人に思わせるにはどうすればいいの,[https://schoo.jp/class/10800],[頭がいい人と思わせる質問の仕方],[30min],安田 正,（株）パンネーションズ・コンサルティング・グループ代表取締役,株式会社パンネーションズ・コンサルティング・グループ代表取締役\n一般社団法人コミュニケーシ...,"[コミュニケーション術, 行動習慣]"
3,ビジネススキル,https://schoo.jp/course/7893,全3回イラストでまなぶ 会計・財務のきほん,"[https://schoo.jp/class/10801, https://schoo.j...","[貸借対照表をブタの貯金箱で読み解く, 損益計算書を風船で表現する]",[60min],松本 めぐみ,Star Compass(株)代表／松本興産(株)取締役,Star Compass株式会社 代表取締役。\n松本興産株式会社 取締役。\n米半導体装置...,"[ビジネス教養, 経理, 会計・財務]"
4,ビジネススキル,https://schoo.jp/course/7873,全1回『うまく言葉にできない』がなくなる言語化力,[https://schoo.jp/class/10763],[『うまく言葉にできない』がなくなる言語化力],[60min],山口 拓朗,伝える力[話す・書く]研究所／山口拓朗ライティングサロン主宰,出版社で編集者・記者を務めたのちに独立。27年間で3700件以上の取材・執筆歴を誇る。現在は...,"[印象づくり, 思考法, 交渉術]"
...,...,...,...,...,...,...,...,...,...,...
877,デザイン,https://schoo.jp/course/7300,全4回デザイン温故知新〜過去から現在、未来へ繋がるヒントを探る,"[https://schoo.jp/class/9425, https://schoo.jp...","[「これまでのデザイン」を理解するための基礎知識, 無印良品など「シンプルなデザイン」の原点...","[60min, 60min, 60min, 60min]",中村 将大,帝京平成大学 助教,帝京平成大学 助教。1983年福岡生まれ。2009年から2021年3月まで東洋美術学校専任講...,"[デザイン, 歴史, リベラルアーツ, デザイン力]"
878,デザイン,https://schoo.jp/course/7344,全5回Figma入門,"[https://schoo.jp/class/9508, https://schoo.jp...","[Figma入門１～インストールして使ってみよう／カラースタイル、テキストスタイル～, Fi...","[30min, 45min, 45min, 30min, 25min]",山本 麻美,株式会社ロクナナ UI/UXデザイナー,インディーズ音楽プラットフォームを作りたい一心でWebを独学。企業からWeb制作の仕事を請け...,"[カンプ制作, スマホサイト制作, プロトタイピング, Figma]"
879,デザイン,https://schoo.jp/course/7265,全1回超実践！ブランディングの現場,[https://schoo.jp/class/9350],[超実践！ブランディングの現場],[60min],西澤 明洋,ブランディングデザイナー,1976年滋賀県生まれ。株式会社エイトブランディングデザイン代表。 「ブランディングデザイン...,"[クリエイティブデザイン, ブランディング, デザイン経営]"
880,デザイン,https://schoo.jp/course/7199,全2回人を動かすバナーデザインの原理原則,"[https://schoo.jp/class/9178, https://schoo.jp...","[人を動かすバナーデザインの原理原則, 実践で学ぶ バナーデザインの「フォントの極意」]","[60min, 60min]",カトウ ヒカル,Webデザイナー,Webデザイン事務所kanvasのWebデザイナー。\nこれまで100サイト以上の制作に関わ...,"[タイポグラフィ, デザイン, ロゴ制作, デザイン力]"


In [34]:
df.head(5)

,category,link,big_title,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,ビジネススキル,https://schoo.jp/course/7887,全1回禅でひも解く「怒り」のしずめかた,[https://schoo.jp/class/10786],[怒った自分を解放する],[60min],河口 智賢,禅僧／耕雲院副住職,曹洞宗耕雲院副住職／曹洞宗観音寺住職／NPO法人ぐんないや-織syoku-理事長／株式会社つ...,"[コミュニケーション術, 行動習慣]"
1,ビジネススキル,https://schoo.jp/course/7876,全1回結果を出す人のチームワーク術,[https://schoo.jp/class/10764],[数字を追うのに疲れたあなたへ〜仕事のやりがいを取り戻すための処方箋〜],[60min],加藤 芳久,経営コンサルタント,●「理念型育成®」を日本で初めて開発・体系化した人財育成の専門家。\n株式会社ファイブベイ、...,"[コミュニケーション術, ファシリテーション, マネジメント, リーダーシップ, 人材育成,..."
2,ビジネススキル,https://schoo.jp/course/7892,全1回頭がいい人に思わせるにはどうすればいいの,[https://schoo.jp/class/10800],[頭がいい人と思わせる質問の仕方],[30min],安田 正,（株）パンネーションズ・コンサルティング・グループ代表取締役,株式会社パンネーションズ・コンサルティング・グループ代表取締役\n一般社団法人コミュニケーシ...,"[コミュニケーション術, 行動習慣]"
3,ビジネススキル,https://schoo.jp/course/7893,全3回イラストでまなぶ 会計・財務のきほん,"[https://schoo.jp/class/10801, https://schoo.j...","[貸借対照表をブタの貯金箱で読み解く, 損益計算書を風船で表現する]",[60min],松本 めぐみ,Star Compass(株)代表／松本興産(株)取締役,Star Compass株式会社 代表取締役。\n松本興産株式会社 取締役。\n米半導体装置...,"[ビジネス教養, 経理, 会計・財務]"
4,ビジネススキル,https://schoo.jp/course/7873,全1回『うまく言葉にできない』がなくなる言語化力,[https://schoo.jp/class/10763],[『うまく言葉にできない』がなくなる言語化力],[60min],山口 拓朗,伝える力[話す・書く]研究所／山口拓朗ライティングサロン主宰,出版社で編集者・記者を務めたのちに独立。27年間で3700件以上の取材・執筆歴を誇る。現在は...,"[印象づくり, 思考法, 交渉術]"


In [36]:
import numpy as np

# カテゴリに応じてIDを割り当てるための条件
conditions = [
    (df['category'] == "ビジネススキル"),
    (df['category'] == "思考術・自己啓発"),
    (df['category'] == "マネジメント"),
    (df['category'] == "マーケティング"),
    (df['category'] == "UI・UXデザイン"),
    (df['category'] == "デザイン")
]

# 各条件に対応するID
values = [1, 2, 3, 4, 5, 6]

# 'category_id'列に条件に応じてIDを割り当てる
df['category_id'] = np.select(conditions, values)

# 結果を表示
df


,category,link,big_title,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words,category_id
0,ビジネススキル,https://schoo.jp/course/7887,全1回禅でひも解く「怒り」のしずめかた,[https://schoo.jp/class/10786],[怒った自分を解放する],[60min],河口 智賢,禅僧／耕雲院副住職,曹洞宗耕雲院副住職／曹洞宗観音寺住職／NPO法人ぐんないや-織syoku-理事長／株式会社つ...,"[コミュニケーション術, 行動習慣]",1
1,ビジネススキル,https://schoo.jp/course/7876,全1回結果を出す人のチームワーク術,[https://schoo.jp/class/10764],[数字を追うのに疲れたあなたへ〜仕事のやりがいを取り戻すための処方箋〜],[60min],加藤 芳久,経営コンサルタント,●「理念型育成®」を日本で初めて開発・体系化した人財育成の専門家。\n株式会社ファイブベイ、...,"[コミュニケーション術, ファシリテーション, マネジメント, リーダーシップ, 人材育成,...",1
2,ビジネススキル,https://schoo.jp/course/7892,全1回頭がいい人に思わせるにはどうすればいいの,[https://schoo.jp/class/10800],[頭がいい人と思わせる質問の仕方],[30min],安田 正,（株）パンネーションズ・コンサルティング・グループ代表取締役,株式会社パンネーションズ・コンサルティング・グループ代表取締役\n一般社団法人コミュニケーシ...,"[コミュニケーション術, 行動習慣]",1
3,ビジネススキル,https://schoo.jp/course/7893,全3回イラストでまなぶ 会計・財務のきほん,"[https://schoo.jp/class/10801, https://schoo.j...","[貸借対照表をブタの貯金箱で読み解く, 損益計算書を風船で表現する]",[60min],松本 めぐみ,Star Compass(株)代表／松本興産(株)取締役,Star Compass株式会社 代表取締役。\n松本興産株式会社 取締役。\n米半導体装置...,"[ビジネス教養, 経理, 会計・財務]",1
4,ビジネススキル,https://schoo.jp/course/7873,全1回『うまく言葉にできない』がなくなる言語化力,[https://schoo.jp/class/10763],[『うまく言葉にできない』がなくなる言語化力],[60min],山口 拓朗,伝える力[話す・書く]研究所／山口拓朗ライティングサロン主宰,出版社で編集者・記者を務めたのちに独立。27年間で3700件以上の取材・執筆歴を誇る。現在は...,"[印象づくり, 思考法, 交渉術]",1
...,...,...,...,...,...,...,...,...,...,...,...
877,デザイン,https://schoo.jp/course/7300,全4回デザイン温故知新〜過去から現在、未来へ繋がるヒントを探る,"[https://schoo.jp/class/9425, https://schoo.jp...","[「これまでのデザイン」を理解するための基礎知識, 無印良品など「シンプルなデザイン」の原点...","[60min, 60min, 60min, 60min]",中村 将大,帝京平成大学 助教,帝京平成大学 助教。1983年福岡生まれ。2009年から2021年3月まで東洋美術学校専任講...,"[デザイン, 歴史, リベラルアーツ, デザイン力]",6
878,デザイン,https://schoo.jp/course/7344,全5回Figma入門,"[https://schoo.jp/class/9508, https://schoo.jp...","[Figma入門１～インストールして使ってみよう／カラースタイル、テキストスタイル～, Fi...","[30min, 45min, 45min, 30min, 25min]",山本 麻美,株式会社ロクナナ UI/UXデザイナー,インディーズ音楽プラットフォームを作りたい一心でWebを独学。企業からWeb制作の仕事を請け...,"[カンプ制作, スマホサイト制作, プロトタイピング, Figma]",6
879,デザイン,https://schoo.jp/course/7265,全1回超実践！ブランディングの現場,[https://schoo.jp/class/9350],[超実践！ブランディングの現場],[60min],西澤 明洋,ブランディングデザイナー,1976年滋賀県生まれ。株式会社エイトブランディングデザイン代表。 「ブランディングデザイン...,"[クリエイティブデザイン, ブランディング, デザイン経営]",6
880,デザイン,https://schoo.jp/course/7199,全2回人を動かすバナーデザインの原理原則,"[https://schoo.jp/class/9178, https://schoo.jp...","[人を動かすバナーデザインの原理原則, 実践で学ぶ バナーデザインの「フォントの極意」]","[60min, 60min]",カトウ ヒカル,Webデザイナー,Webデザイン事務所kanvasのWebデザイナー。\nこれまで100サイト以上の制作に関わ...,"[タイポグラフィ, デザイン, ロゴ制作, デザイン力]",6


In [37]:
# csvファイルに出力する
df.to_csv("schoo_v3.csv", index = False)